In [ ]:
#1. class distribution of test set? - is accuracy even a good metric? - balanced classes - 0.51 fake, 0.49 real
#2. is auc good if class distribution is inequal?

#dataset inc[-no author-], k=10
"""
Comparing the LR model on the test sets

Evaluation Metrics:
Accuracy:  0.621031746031746
Precision:  0.6624489795918367
Recall:  0.5997782705099778
f1-score:  0.62955779674166
"""
 
#count2vect
"""
Accuracy:  0.6421153846153846
Precision:  0.6931993817619784
Recall:  0.6270534778049633
f1-score:  0.658469443934667
"""

#1. add author, dec #features - gives better accuracy with add author+1000 : [worse with l2 and c=1e-4]
"""
Evaluation Metrics:
Accuracy:  0.6078345070422535
Precision:  0.6401225114854517
Recall:  0.5379665379665379
f1-score:  0.5846153846153846
"""
# but without penalty: 
"""
Evaluation Metrics:
Accuracy:  0.6080545774647887
Precision:  0.6401630988786952
Recall:  0.5388245388245388
f1-score:  0.5851385977172141
"""

#2. normalize tfidf output with l2
"""
Evaluation Metrics:
Accuracy:  0.6078345070422535
Precision:  0.6401225114854517
Recall:  0.5379665379665379
f1-score:  0.5846153846153846
"""

#3. shuffle=True in stratifiedkfold
"""
Evaluation Metrics:
Accuracy:  0.6078345070422535
Precision:  0.6401225114854517
Recall:  0.5379665379665379
f1-score:  0.5846153846153846
"""

In [ ]:
! gdown --id 1E8OTtNK_UNSAdmkg3OuB0k2FsKKF9QpY
! gdown --id 1-pm6_kvT9CBrYER40uIkRiG0KMlHnH7L
! gdown --id 1oiVQignVg1wvE1LXKFvOLHPl0P8GNSDp

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
submit_data = pd.read_csv("submit.csv")
test_data = test_data.join(submit_data["label"])

In [ ]:
train_data["author"].fillna("-NO AUTHOR-", inplace = True)
test_data["author"].fillna("-NO AUTHOR-", inplace = True)

In [ ]:
# Removing null values
train_data.dropna(inplace=True)
train_data.drop(train_data.loc[train_data['text']==" "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="  "].index, inplace=True)
train_data.drop(train_data.loc[train_data['text']=="\n"].index, inplace=True)

# Resetting index
train_data.reset_index(inplace=True)
train_data= train_data.apply(lambda x: x.astype(str).str.lower())
train_data = train_data.drop(columns=["id", "index"])

In [ ]:
# Removing null values
test_data.dropna(inplace=True)
test_data.drop(test_data.loc[test_data['text']==" "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="  "].index, inplace=True)
test_data.drop(test_data.loc[test_data['text']=="\n"].index, inplace=True)

# Resetting index
test_data.reset_index(inplace=True)

test_data= test_data.apply(lambda x: x.astype(str).str.lower())
test_data = test_data.drop(columns=["id","index"])

In [ ]:
test_data.describe()

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
! gdown --id 1pGkEyDelD7Lp7yhFML9Dt1oo-91Mqf9H
! gdown --id 1DYVzeYjjTTeHtazHfKD9FTx57h-77bkC

In [ ]:
transformed_df = pd.read_csv("pre_train.csv")
transformed_df_test = pd.read_csv("pre_test.csv")
X_test = transformed_df_test.values
y_test = test_data["label"].values 

In [ ]:
#stratified Kfold (k=5) split for training data

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True)

X = transformed_df.values
y = train_data["label"].values 

X_train=[]
X_val=[]
y_train=[]
y_val=[]

for train_index, test_index in skf.split(X, y):
  X_train.append(X[train_index])
  X_val.append(X[test_index]) 
  y_train.append(y[train_index])
  y_val.append(y[test_index])


add author in preprocessing

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#append author to front of text
#train data

In [ ]:
cols = ['author', 'text']
train_data1 = train_data
train_data1['combined'] = train_data1[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [ ]:
train_data1 = train_data1.drop(columns=["author","text"])

In [ ]:
train_data1.rename(columns = {'combined':'text'}, inplace = True)

In [ ]:
stop_words_set = set(stopwords.words('english'))

sw_removed = []

for i in range(len(train_data1)):
    review = re.sub('[^a-zA-Z]',' ',train_data1['text'][i])
    #not on title?
    review = review.split()
    review = [word for word in review if not word in stop_words_set]
    statements = ' '.join(review)
    sw_removed.append(statements)


In [ ]:
ps = PorterStemmer()

stemmed = []

print(len(sw_removed))

for i in range(len(sw_removed)):
  sw_removed_1 = sw_removed[i].split()
  stem = [ps.stem(word) for word in sw_removed_1]
  st = ' '.join(stem)
  stemmed.append(st)


20126


In [ ]:
preprocess_arr = np.array(stemmed)
#preprocess_df = pd.DataFrame(preprocess_arr)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=100) #3000

train_input = vectorizer.fit_transform(preprocess_arr).toarray()
transformed_df = pd.DataFrame(data=train_input, columns=vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#test set

In [ ]:
cols = ['author', 'text']
test_data1 = test_data
test_data1['combined'] = test_data1[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_data1 = test_data1.drop(columns=["author","text"])
test_data1.rename(columns = {'combined':'text'}, inplace = True)

In [ ]:
stop_words_set = set(stopwords.words('english'))

sw_removed = []

for i in range(len(test_data1)):
    review = re.sub('[^a-zA-Z0-9]',' ',test_data1['text'][i])
    review = review.split()
    review = [word for word in review if not word in stop_words_set]
    statements = ' '.join(review)
    sw_removed.append(statements)


In [ ]:
ps = PorterStemmer()

stemmed = []

print(len(sw_removed))

for i in range(len(sw_removed)):
  sw_removed_1 = sw_removed[i].split()
  stem = [ps.stem(word) for word in sw_removed_1]
  st = ' '.join(stem)
  stemmed.append(st)

5040


In [ ]:
preprocess_arr_test = np.array(stemmed)
test_input = vectorizer.transform(preprocess_arr_test).toarray()
transformed_df_test = pd.DataFrame(data=test_input, columns=vectorizer.get_feature_names())
X_test = transformed_df_test.values
y_test = test_data1["label"].values 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#normalizing using minmax scalar
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(transformed_df)
scaler.transform(transformed_df)
scaler.transform(transformed_df_test)

In [ ]:
X_test = transformed_df_test.values
y_test = test_data1["label"].values 

In [ ]:
#stratified Kfold (k=5) split for training data

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10,shuffle=True)

X = transformed_df.values
y = train_data1["label"].values 

"""X_train=[]
X_val=[]
y_train=[]
y_val=[]

for train_index, test_index in skf.split(X, y):
  X_train.append(X[train_index])
  X_val.append(X[test_index]) 
  y_train.append(y[train_index])
  y_val.append(y[test_index])"""


'X_train=[]\nX_val=[]\ny_train=[]\ny_val=[]\n\nfor train_index, test_index in skf.split(X, y):\n  X_train.append(X[train_index])\n  X_val.append(X[test_index]) \n  y_train.append(y[train_index])\n  y_val.append(y[test_index])'

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_lr_1 = LogisticRegression(random_state=0, C=838, fit_intercept=True, max_iter=760, penalty='l2', tol=0.0003)  

In [ ]:
model_lr_1.fit(X, y.astype('int'))

LogisticRegression(C=838, max_iter=760, random_state=0, tol=0.0003)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
y_pred = model_lr_1.predict(X_test)

print('Comparing the LR model on the test sets')
#model 1
print('\nEvaluation Metrics:')
print("Accuracy: ", accuracy_score( y_test.astype('int'), y_pred))
print("Precision: ", precision_score( y_test.astype('int'), y_pred))
print("Recall: ", recall_score( y_test.astype('int'), y_pred))
print("f1-score: ", f1_score( y_test.astype('int'), y_pred))

Comparing the LR model on the test sets

Evaluation Metrics:
Accuracy:  0.6240079365079365
Precision:  0.6666666666666666
Recall:  0.5994087213599408
f1-score:  0.6312512161899202


In [ ]:
model_lr_1 = LogisticRegression(random_state=0, C=1e-2, fit_intercept=True, max_iter=760, penalty='none', tol=0.0003)  

In [ ]:
model_lr_1.fit(X, y.astype('int'))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


LogisticRegression(C=0.01, max_iter=760, penalty='none', random_state=0,
                   tol=0.0003)

In [ ]:
y_pred = model_lr_1.predict(X_test)

print('Comparing the LR model on the test sets')
#model 1
print('\nEvaluation Metrics:')
print("Accuracy: ", accuracy_score( y_test.astype('int'), y_pred))
print("Precision: ", precision_score( y_test.astype('int'), y_pred))
print("Recall: ", recall_score( y_test.astype('int'), y_pred))
print("f1-score: ", f1_score( y_test.astype('int'), y_pred))

Comparing the LR model on the test sets

Evaluation Metrics:
Accuracy:  0.6080545774647887
Precision:  0.6401630988786952
Recall:  0.5388245388245388
f1-score:  0.5851385977172141
